In [81]:
import PyPDF2
from pathlib import Path
import re

# TO DO
* [ ] wrap **everything** in a `try`
* [ ] add gui
* [ ] test against MAP results
* [ ] add original filename to output
* [ ] pyinstaller

In [82]:
f = "./data/RC_Gr2_S1_2019-20.pdf"
pdf = PyPDF2.PdfFileReader(f, False)
breaks = []
last_record = {'id': None, 'pages': []}
for p in range(0, pdf.numPages):
    this_record = {'id': None, 'pages': []}
    text = pdf.getPage(p).extractText()
    match = re.search('student\W?id:\W?(\d{5,})', text, re.IGNORECASE)
    name = re.search('^student:\W(.*)$', text, re.IGNORECASE)
    if match:
        breaks.append(last_record)
        this_record['id'] = match.group(1)
        this_record['pages'].append(p)
        last_record = this_record
    else:
        last_record['pages'].append(p)

In [79]:
for record in breaks:
    new_pdf = None
    if record['id']:
        new_pdf = PyPDF2.PdfFileWriter()
        for p in record['pages']:
            new_pdf.addPage(pdf.getPage(p))
        
        with open(Path(f'./data/{record["id"]}.pdf'), "wb") as new_file:
            new_pdf.write(new_file)
            